# Analysis of PMAG results

The following document provides the details of the output from PMAG. PMAG outputs the gene orders of ancestral genomes. Here we compute the following for each run of PMAG:
1. The gene content of ancestral genomes.
2. The adjacency content of ancestral genomes.
We then compute the precision recall statistics for each run. We then compare the performance against the ILP results with $\alpha \in \{0, 0.5, 1\}$. 

The PMAG results can be found at "../PMAG_results/mode/MLGOresult_XX/geneorder.out" where mode $\in\{$with_L, without_LT$\}$ and XX $\in \{1,..,20\}$. 

In order to compute precision and recall, these results are compared against the gene orders of the ZOMBI genomes. The ZOMBI gene orders can be found at "../../../sim/mode/Run_XX/G/Genomes/". 

In [1]:
from IPython.display import Image
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [1]:
import os
from collections import defaultdict
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
plt.switch_backend('agg')
%matplotlib inline

In [2]:
def read_file(filename):
    string = open(filename, "r").read()
    string_list = string.split("\n")
    string_list = [line for line in string_list if line and line[0] != '#'] #Read line only if it is nonempty and not a comment.
    return string_list

In [3]:
def get_gene_orders(gene_order_list, orders):
    sp = None
    for line in gene_order_list:
        if line[0] == '>':
            sp = line[1:]
            orders[sp] = []
        else:
            line = line.split(" ")
            if len(line) > 0:
                orders[sp].append(line[:-1])
    return orders

def get_gene_content(genome):
    content = {}
    for chrom in genome:
        for gene in chrom:
            if gene[0] == '-':
                gene = gene[1:]
            if gene in content:
                content[gene] += 1
            else:
                content[gene] = 1
    return content            
          
def get_adjacency_content(genome):
    content = {}
    for chrom in genome:
        i = 0
        for i in range(0, len(chrom) - 1):
            l, r = chrom[i], chrom[i+1]
            #print(l, r)
            l_ext = (l[1:],'t') if l[0] == '-' else (l,'h')
            r_ext = (r[1:],'h') if r[0] == '-' else (r,'t')
            #print(l_ext, r_ext)
            adj = (l_ext, r_ext)
            if adj in content:
                content[adj] += 1
            elif adj[::-1] in content:
                content[adj[::-1]] += 1
            else:
                content[adj] = 1
            i+=1    
    return content            

In [9]:
PMAG_results = "../PMAG_results"
modes = ["with_L", "without_LT", "without_LT_high_rearr"]
PMAG_overall = {}

failed_runs = []
gene_orders = {}
gene_content = {}
adjacency_content = {}
for mode in modes:
    gene_orders[mode] = {}
    gene_content[mode] = {}
    adjacency_content[mode] = {}
    for run in range(1,21):
        gene_orders[mode][run] = {}
        gene_content[mode][run] = {}
        adjacency_content[mode][run] = {}
        gene_order_file = os.path.join(PMAG_results, mode, "MLGOresult_"+str(run), "geneorder.out")
        gene_order_list = read_file(gene_order_file)
        
        if len(gene_order_list) <= 8:
            failed_runs.append((mode, run))
        else:
            gene_orders[mode][run] = get_gene_orders(gene_order_list, gene_orders[mode][run])
            for sp in gene_orders[mode][run]:
                gene_content[mode][run][sp] = {}
                gene_content[mode][run][sp] = get_gene_content(gene_orders[mode][run][sp])
                
                adjacency_content[mode][run][sp] = {}
                adjacency_content[mode][run][sp] = get_adjacency_content(gene_orders[mode][run][sp])
                #print(mode, run, sp, len(adjacency_content[mode][run][sp]))
        #print("\n")        
        


In [10]:
def get_ZOMBI_orders(order_list, order):
    order = []
    for line in order_list:
        line = line.split("\t")
        gfam, orient = line[1], line[2]
        #print(line)
        if orient == "-":
            gene = orient+gfam
        else:
            gene = gfam
        #print(gfam, orient, gene)    
        order.append(gene)
    return order    

In [11]:
ZOMBI_folder = "../../../sim"

ZOMBI_gene_orders = {}
ZOMBI_gene_content = {}
ZOMBI_adjacency_content = {}
for mode in modes:
    ZOMBI_gene_orders[mode] = {}
    ZOMBI_gene_content[mode] = {}
    ZOMBI_adjacency_content[mode] = {}
    for run in range(1,21):
        #print(run)
        A = (mode,run)
        if A not in failed_runs:
            ZOMBI_gene_orders[mode][run] = {}
            ZOMBI_gene_content[mode][run] = {}
            ZOMBI_adjacency_content[mode][run] = {}
            for sp in gene_content[mode][run]:  
                #print(sp)
                gene_order_file = os.path.join(ZOMBI_folder, mode, "Run_"+str(run), "G", "Genomes", sp+"_GENOME.tsv")
                gene_order_list = read_file(gene_order_file)
                #print(gene_order_list)    
                ZOMBI_gene_orders[mode][run][sp] = get_ZOMBI_orders(gene_order_list[1:], gene_orders[mode][run][sp])
                #for sp in gene_orders[mode][run]:
                ZOMBI_gene_content[mode][run][sp] = {}
                ZOMBI_gene_content[mode][run][sp] = get_gene_content(ZOMBI_gene_orders[mode][run][sp])
                #print(ZOMBI_gene_orders[mode][run][sp])
                ZOMBI_adjacency_content[mode][run][sp] = {}
                ZOMBI_adjacency_content[mode][run][sp] = get_adjacency_content([ZOMBI_gene_orders[mode][run][sp]])   



In [12]:
precision, recall, F1_score = {}, {}, {}
avg_prec, avg_rec, avg_F1 = {}, {}, {}
stats = {}
for mode in modes:
    avg_prec[mode], avg_rec[mode], avg_F1[mode] = 0, 0, 0
    stats[mode] = []
    valid_runs = 0
    precision[mode], recall[mode], F1_score[mode] = {}, {}, {}
    for run in range(1,21):
        precision[mode][run], recall[mode][run], F1_score[mode][run] = None, None, None 
        pair = (mode, run)
        if pair not in failed_runs:
            valid_runs += 1
            count = 0
            total_count = 0
            ZOMBI_count = 0
            for sp in adjacency_content[mode][run]:
                for adj in adjacency_content[mode][run][sp]:
                    if adj in ZOMBI_adjacency_content[mode][run][sp]:
                        count += min(adjacency_content[mode][run][sp][adj], ZOMBI_adjacency_content[mode][run][sp][adj])
                        total_count += adjacency_content[mode][run][sp][adj]
                    elif adj[::-1] in ZOMBI_adjacency_content[mode][run][sp]:
                        count += min(adjacency_content[mode][run][sp][adj], ZOMBI_adjacency_content[mode][run][sp][adj[::-1]])
                        total_count += adjacency_content[mode][run][sp][adj]
                    else:
                        total_count += adjacency_content[mode][run][sp][adj]
                for adj in ZOMBI_adjacency_content[mode][run][sp]:
                    ZOMBI_count += ZOMBI_adjacency_content[mode][run][sp][adj]
            precision[mode][run] = count/total_count
            recall[mode][run] = count/ZOMBI_count
            F1_score[mode][run] = 2*precision[mode][run]*recall[mode][run]/(precision[mode][run]+recall[mode][run])
            
            avg_prec[mode] += precision[mode][run]
            avg_rec[mode] += recall[mode][run]
            avg_F1[mode] += F1_score[mode][run]
            
        stats[mode].append([run, precision[mode][run], recall[mode][run], F1_score[mode][run]])
        
        #print(mode, run, precision[mode][run], recall[mode][run], F1_score[mode][run])    
    avg_prec[mode] = avg_prec[mode]/valid_runs
    avg_rec[mode] = avg_rec[mode]/valid_runs
    avg_F1[mode] = avg_F1[mode]/valid_runs
    #print(mode, avg_prec[mode], avg_rec[mode], avg_F1[mode])
    stats[mode].append(["Overall", avg_prec[mode], avg_rec[mode], avg_F1[mode]])
    PMAG_overall[mode] = ["PMAG", avg_prec[mode], avg_rec[mode], avg_F1[mode]]
    stats[mode] = pd.DataFrame(stats[mode])
    stats[mode].rename(columns = {0: 'Run', 1: 'Precision', 2: 'Recall', 3: 'F1 score'}, inplace = True)
    #stats[mode] = stats[mode].sort_values(by=['Run'])

### With losses

PMAG/MLGO was implemented to completion on 18 of the 20 inputs. It performs better than the SPP ILP for the most part with a recall of at least 0.7 consistently and a precision of at least 0.75 for most of the runs. The SPP ILP outperforms PMAG in precision, having a precision of at least 0.9 for lower values of $\alpha$. Considering the F1 score, only the case $\alpha=0.5$ (F1 score $= 0.78$) is able to provide some competition for PMAG when gene losses are considered.

In [13]:
stats['with_L']

,Run,Precision,Recall,F1 score
0,1,0.077626,0.083641,0.080521
1,2,0.090566,0.093023,0.091778
2,3,0.088028,0.102041,0.094518
3,4,0.125436,0.134663,0.129886
4,5,0.135405,0.137698,0.136542
5,6,0.881563,0.888069,0.884804
6,7,0.829268,0.826087,0.827675
7,8,0.864407,0.867558,0.865979
8,9,0.898876,0.893300,0.896080
9,10,0.884041,0.873016,0.878494


### Without losses

Once again PMAG/MLGO was implemented to completion for 18 out of 20 inputs. PMAG consistently provided a precision of 0.74 and recall of 0.8. However, in this case, the SPP ILP was able to provide better statistics for $\alpha=0.5$ and $0.75$. While the recall for $\alpha=0.75$ was better than other choices for the parameter, the average F1 scores for $\alpha \in \{0.25, 0.5, 0.75\}$ was better than that for PMAG. 

In [14]:
stats['without_LT']

,Run,Precision,Recall,F1 score
0,1,NaN,NaN,NaN
1,2,0.764548,0.816444,0.789644
2,3,0.792982,0.832413,0.812219
3,4,0.805629,0.841912,0.823371
4,5,0.801105,0.838150,0.819209
5,6,0.788445,0.831541,0.809420
6,7,0.793814,0.847706,0.819876
7,8,0.747547,0.808880,0.777005
8,9,0.795477,0.829744,0.812249
9,10,0.805683,0.836346,0.820728


In [15]:
stats['without_LT_high_rearr']

,Run,Precision,Recall,F1 score
0,1,0.008264,0.008604,0.008431
1,2,0.010516,0.010628,0.010572
2,3,0.009634,0.010121,0.009872
3,4,0.008547,0.008555,0.008551
4,5,0.016791,0.017946,0.017349
5,6,0.007000,0.007431,0.007209
6,7,0.005666,0.005650,0.005658
7,8,0.013672,0.013527,0.013599
8,9,0.007851,0.008016,0.007933
9,10,0.013035,0.014403,0.013685


# Weighted adjacencies ILP results

In this experiment, we will use the following as input.
1. Species tree (generated by ZOMBI)
2. Gene order for extant genomes (generated by ZOMBI)
3. Gene content for ancestral genomes (obtained through ancestral gene orders generated by ZOMBI)
4. Reconciled gene trees for each gene family (generated by ZOMBI)

Here, we use DeClone for sampling and obtaining adjacency weights. This experiment adds some noise to the reconstruction by using a larger set of potential adjacencies with non-uniform weights.

In this experiment, the species tree topology is the same for all runs of the ILP.

### Experiment set up

#### Simulations

We use two sets of simulations:
1. Without loss and low rates of rearrangement (../../../sim/without_LT)
2. With gene loss as an event and low rates of rearrangment (../../../sim/with_L)

For each set of simulations, we have 10 extant genomes. The Root genome contains 100 gene families. Each set has 200 duplications over all branches of the species tree. The rate of rearrangement for set 1 is 100 inversions and 100 translocations over all branches of the species tree. For set 2, the rate of gene loss is 100 genes over all branches. The exact parameters for the sets can be found under the directory "../../../code/ZOMBI_old" in the folders "no_loss_params" and "with_loss_params" respectively.  

While generating the species tree, the option for extinction of species has been muted. Also, the option for horizontal gene transfer has also been muted for this experiment.

The adjacencies have been obtained through two runs of DeClone, one with temperatue 0.1 and another with temperature 1 in order to observe the effects of the number of candidate adjacencies. 

### ILP

The ILP is run with the linearization parameter $\alpha \in \{0, 0.25, 0.5, 0.75, 1\}$ for each of the 20 runs. For each combination (Run, $\alpha$), we compare the adjacency sets provided by the ILP to the true adjacencies from ZOMBI genomes. We generate precision-recall statistics and compare them for each $\alpha$ value. For one solution selected by the ILP, we also output the gene order for each species and the cuts and joins involved for the solution.

In [17]:
with_L_1 = "../output/with_L/temp=1"
with_L_01 = "../output/with_L/temp=0.1"
no_LT_01 = "../output/without_LT/temp=0.1"
no_LT_1 = "../output/without_LT/temp=1"
no_LT_hr_01 = "../output/without_LT_high_rearr/temp=0.1"
no_LT_hr_1 = "../output/without_LT_high_rearr/temp=1"

prec, rec, F1 = {}, {}, {}
dist, cuts, joins , dups, obs = {}, {}, {}, {}, {}
b_dist, b_cuts, b_joins , b_dups, b_obs = {}, {}, {}, {}, {}

d_per_branch = {}
dist_scores = {}

mean = {}
mean_scores = {}

In [18]:
def update_dict(line, stat_dict, alpha):
    stat = line.split(" ")[-1]
    stat_dict[alpha].append(float(stat))
    return stat_dict

def append_dist(line, d1, d2, d3, d4, d5, alpha):
    #s1, s2, s3, s4, s5 = line.split("\t")[1], line.split("\t")[2], line.split("\t")[3], line.split("\t")[4], line.split("\t")[5]
    s1, s2, s3 = line.split("\t")[1], line.split("\t")[2], line.split("\t")[3]
    d1[alpha].append(float(s1))
    d2[alpha].append(float(s2))
    d3[alpha].append(float(s3))
    #d4[alpha].append(float(s4))
    #d5[alpha].append(float(s5))
    return d1, d2, d3, d4, d5

def append_b_dist(line, d1, d2, d3, d4, d5, alpha):
    b = line.split("\t")[0]
    #print(b)
    s1, s2, s3, s4, s5 = line.split("\t")[1], line.split("\t")[2], line.split("\t")[3], line.split("\t")[4], line.split("\t")[5]

    if b not in d1[alpha]:
        d1[alpha][b], d2[alpha][b], d3[alpha][b], d4[alpha][b], d5[alpha][b] = {}, {}, {}, {}, {}
        d1[alpha][b], d2[alpha][b], d3[alpha][b], d4[alpha][b], d5[alpha][b] = [], [], [], [], []
    d1[alpha][b].append(float(s1))
    d2[alpha][b].append(float(s2))
    d3[alpha][b].append(float(s3))
    d4[alpha][b].append(float(s4))
    d5[alpha][b].append(float(s5))
    return d1, d2, d3, d4, d5


In [19]:
def get_stats(case):
    
    idx = case.split("/")[-1]
    print(idx)
    prec[idx], rec[idx], F1[idx] = defaultdict(list), defaultdict(list), defaultdict(list)
    dist[idx], cuts[idx], joins[idx], dups[idx], obs[idx] = defaultdict(list), defaultdict(list), defaultdict(list), defaultdict(list), defaultdict(list)
    b_dist[idx], b_cuts[idx], b_joins[idx], b_dups[idx], b_obs[idx] = defaultdict(dict), defaultdict(dict), defaultdict(dict), defaultdict(dict), defaultdict(dict)

    files = [os.path.join(dp, f) for dp, dn, filenames in os.walk(case) for f in filenames if "stats" in f]
    for file in files:
        #print(file.split("/")[4].split("_"))
        run = file.split("/")[4].split("_")[1]
        alpha = file.split("/")[4].split("_")[-1]
        with open(file, 'r') as f:
            for line in f:
                if "Precision" in line:
                    prec[idx] = update_dict(line, prec[idx], alpha)
                elif "Recall" in line:
                    rec[idx] = update_dict(line, rec[idx], alpha)
                elif "F1_score:" in line:
                    F1[idx] = update_dict(line, F1[idx], alpha)
                elif "(" in line and "None" not in line:
                    if len(line.split("\t")) > 3:
                        b_dist[idx], b_cuts[idx], b_joins[idx], b_dups[idx], b_obs[idx] = append_b_dist(line, b_dist[idx], b_cuts[idx], b_joins[idx], b_dups[idx], b_obs[idx], alpha)

                elif "Overall" in line:
                    dist[idx], cuts[idx], joins[idx], dups[idx], obs[idx] = append_dist(line, dist[idx], cuts[idx], joins[idx], dups[idx], obs[idx], alpha)

    d_per_branch[idx] = {}
    dist_scores[idx] = {}
    for alpha in b_dist[idx]:
        d_per_branch[idx][alpha] = {}
        dist_scores[idx][alpha] = []
        for branch in b_dist[idx][alpha]:
            #print(branch)
            d_per_branch[idx][alpha][branch] = {}
            d_per_branch[idx][alpha][branch]['dist'] = sum(b_dist[idx][alpha][branch])/len(b_dist[idx][alpha][branch])
            d_per_branch[idx][alpha][branch]['cuts'] = sum(b_cuts[idx][alpha][branch])/len(b_dist[idx][alpha][branch])
            d_per_branch[idx][alpha][branch]['joins'] = sum(b_joins[idx][alpha][branch])/len(b_dist[idx][alpha][branch])            
            d_per_branch[idx][alpha][branch]['dups'] = sum(b_dups[idx][alpha][branch])/len(b_dist[idx][alpha][branch])
            d_per_branch[idx][alpha][branch]['observed_dups'] = sum(b_obs[idx][alpha][branch])/len(b_dist[idx][alpha][branch])
            dist_scores[idx][alpha].append([branch, d_per_branch[idx][alpha][branch]['dist'], d_per_branch[idx][alpha][branch]['cuts'], d_per_branch[idx][alpha][branch]['joins'],d_per_branch[idx][alpha][branch]['dups'],d_per_branch[idx][alpha][branch]['observed_dups']])
        dist_scores[idx][alpha] = pd.DataFrame(dist_scores[idx][alpha]) 
        dist_scores[idx][alpha].rename(columns = {0: 'branch', 1: 'SCJTDFD', 2: 'Cuts', 3: 'Joins',4:'Dups',5:'Observed'}, inplace = True) 

    mean[idx] = {}
    mean_scores[idx] = []
    for alpha in prec[idx]:
        mean[idx][alpha] = {}
        mean[idx][alpha]['precision'] = sum(prec[idx][alpha])/len(prec[idx][alpha])
        mean[idx][alpha]['recall'] = sum(rec[idx][alpha])/len(rec[idx][alpha])
        mean[idx][alpha]['f1_score'] = sum(F1[idx][alpha])/len(F1[idx][alpha])
        mean_scores[idx].append([alpha, mean[idx][alpha]['precision'], mean[idx][alpha]['recall'], mean[idx][alpha]['f1_score']])

    mean_scores[idx] = pd.DataFrame(mean_scores[idx])
    mean_scores[idx].rename(columns = {0: 'alpha', 1: 'Precision', 2: 'Recall', 3: 'F1 score'}, inplace = True) 
    mean_scores[idx] = mean_scores[idx].sort_values(by=['alpha'])
    
    #print(mean_scores)
    #mean_scores[idx]

## No Loss and Transfer

### Mean statistics for temp = 1

The following table lists the average precision, recall and F1 score over 20 runs. The precision is best for lower values of $\alpha$. On the other hand the recall is progressively lower for lower values of $\alpha$. The best performance occurs at $\alpha=0.5$ with an F1-score of 0.87 followed closely by $\alpha=0.75$ with an F1-score of 0.86. With the exception of $\alpha=1$ the precision is consistently above 0.9.

In [20]:
get_stats(no_LT_1)
idx = no_LT_1.split("/")[-1]

temp=1


In [21]:
prec_1, rec_1, F1_1 = PMAG_overall["without_LT"][1], PMAG_overall["without_LT"][2], PMAG_overall["without_LT"][3]
mean_scores[idx].append({"alpha":'PMAG', "Precision":prec_1, "Recall":rec_1, "F1 score": F1_1},ignore_index=True)

,alpha,Precision,Recall,F1 score
0,0,0.974026,0.055522,0.104861
1,0.25,0.950688,0.071409,0.132618
2,0.5,0.950613,0.071489,0.132754
3,0.75,0.824759,0.622082,0.708914
4,1,0.747711,0.728392,0.737914
5,PMAG,0.790318,0.837004,0.812954


### Branch wise distance for $\alpha=0$ and temp $=1$ for without_LT runs

In [22]:
dist_scores[idx][str(0)].sort_values(by=['branch'])

,branch,SCJTDFD,Cuts,Joins,Dups,Observed
5,"(Root,n1)",192.60,0.90,0.00,0.0,0.0
13,"(Root,n2)",303.10,0.00,0.00,0.0,0.0
8,"(n1,n3)",72.70,0.00,0.00,0.0,0.0
16,"(n1,n4)",138.20,0.00,0.00,0.0,0.0
7,"(n12,n15)",296.00,0.35,2.25,0.0,0.0
11,"(n12,n16)",295.85,0.25,2.10,0.0,0.0
9,"(n3,n5)",305.10,0.00,1.70,0.0,0.0
10,"(n3,n6)",268.80,0.50,0.00,0.0,0.0
15,"(n4,n7)",214.40,0.00,0.00,0.0,0.0
3,"(n4,n8)",178.00,0.00,0.00,0.0,0.0


### Branch wise distance for $\alpha=0.25$ and temp $=1$ for without_LT runs

In [23]:
dist_scores[idx][str(0.25)].sort_values(by=['branch'])

,branch,SCJTDFD,Cuts,Joins,Dups,Observed
5,"(Root,n1)",192.60,0.90,0.00,0.0,0.0
13,"(Root,n2)",303.10,0.00,0.00,0.0,0.0
8,"(n1,n3)",72.70,0.00,0.00,0.0,0.0
16,"(n1,n4)",138.20,0.00,0.00,0.0,0.0
7,"(n12,n15)",295.95,0.95,1.60,0.0,0.0
11,"(n12,n16)",295.90,0.90,1.50,0.0,0.0
9,"(n3,n5)",303.80,0.00,0.40,0.0,0.0
10,"(n3,n6)",270.10,1.80,0.00,0.0,0.0
15,"(n4,n7)",214.40,0.00,0.00,0.0,0.0
3,"(n4,n8)",178.00,0.00,0.00,0.0,0.0


### Branch wise distance for $\alpha=0.5$ and temp $=1$ for without_LT runs

In [24]:
dist_scores[idx][str(0.5)].sort_values(by=['branch'])

,branch,SCJTDFD,Cuts,Joins,Dups,Observed
5,"(Root,n1)",192.55,0.85,0.00,0.0,0.0
13,"(Root,n2)",303.10,0.00,0.00,0.0,0.0
8,"(n1,n3)",72.75,0.05,0.00,0.0,0.0
16,"(n1,n4)",138.25,0.05,0.00,0.0,0.0
7,"(n12,n15)",295.95,0.95,1.60,0.0,0.0
11,"(n12,n16)",295.90,0.90,1.50,0.0,0.0
9,"(n3,n5)",303.80,0.00,0.40,0.0,0.0
10,"(n3,n6)",270.10,1.80,0.00,0.0,0.0
15,"(n4,n7)",214.40,0.00,0.00,0.0,0.0
3,"(n4,n8)",178.00,0.00,0.00,0.0,0.0


### Branch wise distance for $\alpha=0.75$ and temp $=1$ for without_LT runs

In [25]:
dist_scores[idx][str(0.75)].sort_values(by=['branch'])

,branch,SCJTDFD,Cuts,Joins,Dups,Observed
5,"(Root,n1)",267.25,75.95,0.00,0.0,0.20
13,"(Root,n2)",379.10,75.95,0.05,0.0,0.00
8,"(n1,n3)",105.70,33.10,0.00,0.0,0.05
16,"(n1,n4)",188.05,50.05,0.00,0.0,0.10
7,"(n12,n15)",404.15,109.15,1.60,0.0,0.00
11,"(n12,n16)",404.20,109.15,1.55,0.0,0.00
9,"(n3,n5)",385.15,81.35,0.40,0.0,0.00
10,"(n3,n6)",349.35,81.10,0.05,0.0,0.05
15,"(n4,n7)",296.00,81.80,0.00,0.0,0.10
3,"(n4,n8)",256.40,78.55,0.05,0.0,0.10


### Branch wise distance for $\alpha=1$ and temp $=1$ for without_LT runs

In [26]:
dist_scores[idx][str(1)].sort_values(by=['branch'])

,branch,SCJTDFD,Cuts,Joins,Dups,Observed
5,"(Root,n1)",819.70,627.95,0.25,0.0,0.10
13,"(Root,n2)",931.30,627.95,0.25,0.0,0.00
8,"(n1,n3)",782.80,690.35,20.05,0.0,0.15
16,"(n1,n4)",837.05,690.35,9.20,0.0,0.35
7,"(n12,n15)",2075.10,1779.75,1.95,0.0,0.00
11,"(n12,n16)",2075.40,1779.75,2.15,0.0,0.00
9,"(n3,n5)",1060.50,756.10,1.00,0.0,0.00
10,"(n3,n6)",1025.85,755.95,1.60,0.0,0.00
15,"(n4,n7)",1383.80,1162.60,6.80,0.0,0.00
3,"(n4,n8)",1351.00,1163.00,10.10,0.0,0.05


### Mean statistics for temp=0.1

The performance in presence of limited candidate adjacencies is very poor. Although the precision is always above 0.9, the recall is very low. The performance is increases gradually as we increase $\alpha$ albeit by a very small margin.

In [27]:
get_stats(no_LT_01)
idx = no_LT_01.split("/")[-1]

temp=0.1


In [28]:
prec_01, rec_01, F1_01 = PMAG_overall["without_LT"][1], PMAG_overall["without_LT"][2], PMAG_overall["without_LT"][3]
mean_scores[idx].append({"alpha":'PMAG', "Precision":prec_01, "Recall":rec_01, "F1 score": F1_01},ignore_index=True)

,alpha,Precision,Recall,F1 score
0,0,0.995199,0.036459,0.070248
1,0.25,0.987744,0.040028,0.076825
2,0.5,0.989104,0.045110,0.086165
3,0.75,0.986268,0.349558,0.514846
4,1,0.901514,0.377253,0.530840
5,PMAG,0.790318,0.837004,0.812954


### Branch wise distance for $\alpha=0$ and temp $=0.1$ for without_LT runs

In [29]:
dist_scores[idx][str(0)].sort_values(by=['branch'])

,branch,SCJTDFD,Cuts,Joins,Dups,Observed
5,"(Root,n1)",192.10,0.40,0.00,0.0,0.0
13,"(Root,n2)",303.60,0.00,0.50,0.0,0.0
8,"(n1,n3)",72.70,0.00,0.00,0.0,0.0
16,"(n1,n4)",138.20,0.00,0.00,0.0,0.0
7,"(n12,n15)",297.15,0.15,3.60,0.0,0.0
11,"(n12,n16)",297.40,0.20,3.70,0.0,0.0
9,"(n3,n5)",305.00,0.00,1.60,0.0,0.0
10,"(n3,n6)",268.70,0.40,0.00,0.0,0.0
15,"(n4,n7)",214.40,0.00,0.00,0.0,0.0
3,"(n4,n8)",178.00,0.00,0.00,0.0,0.0


### Branch wise distance for $\alpha=0.25$ and temp $=0.1$ for without_LT runs

In [30]:
dist_scores[idx][str(0.25)].sort_values(by=['branch'])

,branch,SCJTDFD,Cuts,Joins,Dups,Observed
5,"(Root,n1)",192.45,0.75,0.00,0.0,0.0
13,"(Root,n2)",303.25,0.00,0.15,0.0,0.0
8,"(n1,n3)",72.70,0.00,0.00,0.0,0.0
16,"(n1,n4)",138.20,0.00,0.00,0.0,0.0
7,"(n12,n15)",297.15,0.25,3.50,0.0,0.0
11,"(n12,n16)",297.40,0.30,3.60,0.0,0.0
9,"(n3,n5)",304.90,0.00,1.50,0.0,0.0
10,"(n3,n6)",268.80,0.50,0.00,0.0,0.0
15,"(n4,n7)",214.40,0.00,0.00,0.0,0.0
3,"(n4,n8)",178.00,0.00,0.00,0.0,0.0


### Branch wise distance for $\alpha=0.5$ and temp $=0.1$ for without_LT runs

In [31]:
dist_scores[idx][str(0.5)].sort_values(by=['branch'])

,branch,SCJTDFD,Cuts,Joins,Dups,Observed
5,"(Root,n1)",192.30,0.60,0.00,0.0,0.0
13,"(Root,n2)",303.25,0.00,0.15,0.0,0.0
8,"(n1,n3)",72.85,0.15,0.00,0.0,0.0
16,"(n1,n4)",138.35,0.15,0.00,0.0,0.0
7,"(n12,n15)",302.50,5.60,3.50,0.0,0.0
11,"(n12,n16)",302.75,5.65,3.60,0.0,0.0
9,"(n3,n5)",304.90,0.00,1.50,0.0,0.0
10,"(n3,n6)",268.55,0.25,0.00,0.0,0.0
15,"(n4,n7)",214.40,0.00,0.00,0.0,0.0
3,"(n4,n8)",178.00,0.00,0.00,0.0,0.0


### Branch wise distance for $\alpha=0.75$ and temp $=0.1$ for without_LT runs

In [32]:
dist_scores[idx][str(0.75)].sort_values(by=['branch'])

,branch,SCJTDFD,Cuts,Joins,Dups,Observed
5,"(Root,n1)",217.60,25.90,0.00,0.0,0.0
13,"(Root,n2)",328.60,25.35,0.15,0.0,0.0
8,"(n1,n3)",83.85,7.10,4.05,0.0,0.0
16,"(n1,n4)",152.10,11.10,2.80,0.0,0.0
7,"(n12,n15)",376.05,79.15,3.50,0.0,0.0
11,"(n12,n16)",376.30,79.20,3.60,0.0,0.0
9,"(n3,n5)",323.95,19.05,1.50,0.0,0.0
10,"(n3,n6)",287.45,18.90,0.25,0.0,0.0
15,"(n4,n7)",233.45,17.45,1.60,0.0,0.0
3,"(n4,n8)",197.15,17.85,1.30,0.0,0.0


### Branch wise distance for $\alpha=1$ and temp $=0.1$ for without_LT runs

In [33]:
dist_scores[idx][str(1)].sort_values(by=['branch'])

,branch,SCJTDFD,Cuts,Joins,Dups,Observed
5,"(Root,n1)",312.80,121.10,0.00,0.0,0.0
13,"(Root,n2)",424.55,121.25,0.20,0.0,0.0
8,"(n1,n3)",113.00,33.00,7.30,0.0,0.0
16,"(n1,n4)",175.15,32.80,4.15,0.0,0.0
7,"(n12,n15)",385.20,88.30,3.50,0.0,0.0
11,"(n12,n16)",385.60,88.50,3.60,0.0,0.0
9,"(n3,n5)",341.30,36.40,1.50,0.0,0.0
10,"(n3,n6)",302.35,33.80,0.25,0.0,0.0
15,"(n4,n7)",246.90,30.10,2.40,0.0,0.0
3,"(n4,n8)",207.60,27.80,1.80,0.0,0.0


## With loss

When gene loss is allowed as an event, the recall takes a significant drop as compared to the previous case. The precision however is still above 90% for lower values of $\alpha$. The F1 score is the highest for $\alpha=0.5$ and gets progressively power towards either direction of $\alpha=0.5$.

In [34]:
get_stats(with_L_1)
idx = with_L_1.split("/")[-1]

temp=1


In [35]:
prec_1, rec_1, F1_1 = PMAG_overall["with_L"][1], PMAG_overall["with_L"][2], PMAG_overall["with_L"][3]
mean_scores[idx].append({"alpha":'PMAG', "Precision":prec_1, "Recall":rec_1, "F1 score": F1_1},ignore_index=True)

,alpha,Precision,Recall,F1 score
0,0,0.986904,0.043319,0.081838
1,0.25,0.930936,0.060749,0.111508
2,0.5,0.931306,0.061099,0.112152
3,0.75,0.817500,0.294679,0.389546
4,1,0.488630,0.337052,0.385178
5,PMAG,0.620390,0.612789,0.616466


In [36]:
get_stats(with_L_01)
idx = with_L_01.split("/")[-1]

temp=0.1


In [37]:
prec_01, rec_01, F1_01 = PMAG_overall["with_L"][1], PMAG_overall["with_L"][2], PMAG_overall["with_L"][3]
mean_scores[idx].append({"alpha":'PMAG', "Precision":prec_01, "Recall":rec_01, "F1 score": F1_01},ignore_index=True)

,alpha,Precision,Recall,F1 score
0,0,0.986042,0.060103,0.112973
1,0.25,0.968623,0.065978,0.123143
2,0.5,0.973012,0.073983,0.136966
3,0.75,0.877934,0.482991,0.620343
4,1,0.838676,0.507967,0.630278
5,PMAG,0.620390,0.612789,0.616466


## Without loss but high rearrangement

In [38]:
get_stats(no_LT_hr_1)
idx = no_LT_hr_1.split("/")[-1]

temp=1


In [39]:
mean_scores[idx]

,alpha,Precision,Recall,F1 score
3,0,0.542462,0.011266,0.022051
1,0.25,0.453087,0.025375,0.047975
2,0.5,0.452519,0.025333,0.047896
4,0.75,0.644830,0.336755,0.442244
0,1,0.445122,0.441152,0.443128


In [40]:
get_stats(no_LT_hr_01)
idx = no_LT_hr_01.split("/")[-1]

temp=0.1


In [41]:
mean_scores[idx]

,alpha,Precision,Recall,F1 score
3,0,0.990598,0.009044,0.017905
2,0.25,0.974487,0.010204,0.020178
1,0.5,0.981407,0.014743,0.028990
4,0.75,0.985269,0.181186,0.305745
0,1,0.969333,0.187561,0.313932
